# Sheet properties

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

# Imports

In [2]:
import sys
sys.path.append('/home/rcendre/classification')
from numpy import logspace
from scipy.stats import randint,uniform
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from toolbox.classification.common import IO, Tools
from toolbox.classification.parameters import Settings
from toolbox.transforms.labels import OrderedEncoder
from toolbox.views.common import Views, ViewsTools
import warnings
warnings.filterwarnings('ignore') 

# Parameters

In [3]:
# Advanced parameters
validation = 4
settings = Settings.get_default_dermatology()
max_iter = -1

# Inputs

In [4]:
features_file = f'Dermoscopy.pickle'
prediction_file = f'Prediction_Transfer.pickle'

In [5]:
inputs = IO.load(features_file)

# Features extraction

In [6]:
extractors = [('VGG16Max', {}), ('InceptionV3Max', {}), ('InceptionResNetV2Max', {}), ('ResNetMax', {}),
              ('VGG16Avg', {}), ('InceptionV3Avg', {}), ('InceptionResNetV2Avg', {}), ('ResNetAvg', {})]

# Models

In [7]:
clfs = []

# SVM Linear
clf = ('clf', SVC(kernel='linear', class_weight='balanced', max_iter=max_iter, probability=True))
clf_params = {'clf__C': logspace(-2, 3, 6).tolist()}
clfs.append(('SVML', clf, clf_params))


In [8]:
scalers = [('MMS',('scale', MinMaxScaler())), ('SS',('scale', StandardScaler()))]

processes = []
for scaler_name,scaler in scalers:

    for clf_name, clf, clf_params in clfs:
    
        process = Pipeline([scaler, clf])
        processes.append((f'{scaler_name}_{clf_name}', process, clf_params))

# Evaluation

In [9]:
for extractor in extractors: 
    # Decompose model
    extractor_name, extractor_process = extractor
    
    for model in processes:   
        # Decompose model       
        model_name, model_process, model_params = model
        
        # Start evaluation
        name = f'{extractor_name}{model_name}Label'        
        print(f'{name} performed...', end='\r')
        Tools.evaluate(inputs, {'datum': extractor_name, 'label_encode': 'LesionEncode'}, model_process, name, distribution=model_params)
        print(f'{name} achieved !', end='\r')  
        
        # Save
        IO.save(inputs, prediction_file)

# Scores and ROC

In [10]:
from IPython.display import HTML
from IPython.display import display

inputs = IO.load(prediction_file)

# Transform labels
label_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))
for extractor in extractors: 
    extractor_name, extractor_process = extractor
    
    for model in processes:  
        # Decompose model
        model_name, model_process, model_params = model        
        
        # Label
        name = f'{extractor_name}{model_name}Label' 
        display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, label_encoder)],
                                                                        title=[f'Test - {name}'])))

,precision,recall,f1-score,support
Benign,0.53±0.10,0.63±0.12,0.57±0.02,86.00±6.18
Malignant,0.74±0.10,0.65±0.11,0.69±0.05,138.00±4.56
accuracy,0.64±0.03,0.64±0.03,0.64±0.03,0.64±0.03
macro avg,0.63±0.01,0.64±0.02,0.63±0.02,224.00±4.30
weighted avg,0.66±0.03,0.64±0.03,0.65±0.04,224.00±4.30


,precision,recall,f1-score,support
Benign,0.55±0.09,0.62±0.10,0.58±0.05,86.00±6.18
Malignant,0.74±0.10,0.68±0.08,0.71±0.05,138.00±4.56
accuracy,0.66±0.04,0.66±0.04,0.66±0.04,0.66±0.04
macro avg,0.64±0.03,0.65±0.04,0.64±0.03,224.00±4.30
weighted avg,0.67±0.04,0.66±0.04,0.66±0.04,224.00±4.30


,precision,recall,f1-score,support
Benign,0.55±0.12,0.56±0.13,0.55±0.05,86.00±6.18
Malignant,0.72±0.11,0.72±0.08,0.72±0.02,138.00±4.56
accuracy,0.66±0.03,0.66±0.03,0.66±0.03,0.66±0.03
macro avg,0.64±0.02,0.64±0.03,0.64±0.03,224.00±4.30
weighted avg,0.66±0.03,0.66±0.03,0.66±0.04,224.00±4.30


,precision,recall,f1-score,support
Benign,0.53±0.10,0.49±0.14,0.51±0.05,86.00±6.18
Malignant,0.70±0.11,0.73±0.06,0.71±0.04,138.00±4.56
accuracy,0.64±0.04,0.64±0.04,0.64±0.04,0.64±0.04
macro avg,0.61±0.03,0.61±0.05,0.61±0.04,224.00±4.30
weighted avg,0.63±0.06,0.64±0.04,0.64±0.05,224.00±4.30


,precision,recall,f1-score,support
Benign,0.52±0.13,0.55±0.17,0.53±0.05,86.00±6.18
Malignant,0.71±0.12,0.69±0.13,0.70±0.06,138.00±4.56
accuracy,0.63±0.06,0.63±0.06,0.63±0.06,0.63±0.06
macro avg,0.62±0.05,0.62±0.05,0.62±0.05,224.00±4.30
weighted avg,0.64±0.07,0.63±0.06,0.64±0.06,224.00±4.30


...

,precision,recall,f1-score,support
Benign,0.58±0.16,0.62±0.09,0.60±0.07,86.00±6.18
Malignant,0.75±0.08,0.72±0.11,0.73±0.03,138.00±4.56
accuracy,0.68±0.05,0.68±0.05,0.68±0.05,0.68±0.05
macro avg,0.66±0.06,0.67±0.04,0.66±0.05,224.00±4.30
weighted avg,0.68±0.03,0.68±0.05,0.68±0.04,224.00±4.30


,precision,recall,f1-score,support
Benign,0.60±0.19,0.60±0.10,0.60±0.04,86.00±6.18
Malignant,0.75±0.08,0.75±0.13,0.75±0.02,138.00±4.56
accuracy,0.69±0.03,0.69±0.03,0.69±0.03,0.69±0.03
macro avg,0.67±0.05,0.68±0.02,0.68±0.03,224.00±4.30
weighted avg,0.69±0.03,0.69±0.03,0.69±0.02,224.00±4.30


,precision,recall,f1-score,support
Benign,0.64±0.16,0.62±0.11,0.63±0.05,86.00±6.18
Malignant,0.77±0.10,0.78±0.11,0.77±0.04,138.00±4.56
accuracy,0.72±0.04,0.72±0.04,0.72±0.04,0.72±0.04
macro avg,0.70±0.05,0.70±0.03,0.70±0.04,224.00±4.30
weighted avg,0.72±0.02,0.72±0.04,0.72±0.04,224.00±4.30


,precision,recall,f1-score,support
Benign,0.59±0.17,0.63±0.12,0.61±0.04,86.00±6.18
Malignant,0.76±0.10,0.72±0.14,0.74±0.04,138.00±4.56
accuracy,0.69±0.04,0.69±0.04,0.69±0.04,0.69±0.04
macro avg,0.67±0.05,0.68±0.02,0.67±0.04,224.00±4.30
weighted avg,0.69±0.02,0.69±0.04,0.69±0.04,224.00±4.30


,precision,recall,f1-score,support
Benign,0.62±0.19,0.64±0.12,0.63±0.10,86.00±6.18
Malignant,0.77±0.10,0.76±0.14,0.77±0.07,138.00±4.56
accuracy,0.71±0.08,0.71±0.08,0.71±0.08,0.71±0.08
macro avg,0.70±0.08,0.70±0.07,0.70±0.08,224.00±4.30
weighted avg,0.72±0.06,0.71±0.08,0.71±0.08,224.00±4.30
